In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [44]:
load_dotenv()

True

In [45]:
api_key_pw = os.getenv('api_key_bot')
api_secret_pw = os.getenv('api_secret_bot')
mail_key = os.getenv("mail_key")
mail = os.getenv("mail")


In [46]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)
frame = pd.DataFrame(session_auth.query_kline(symbol="MATICUSDT", interval="5m")["result"])
frame


,0,1,2,3,4,5,6,7,8,9,10
0,1671428100000,0.8024,0.8024,0.801,0.8021,9988.08,1671428400000,8006.169699,38,0,0
1,1671428400000,0.8021,0.8021,0.8004,0.8009,15099.5,1671428700000,12096.337061,50,0,0
2,1671428700000,0.8009,0.8028,0.8005,0.8028,6295.01,1671429000000,5046.687242,26,0,0
3,1671429000000,0.8028,0.8028,0.8003,0.8003,3865.04,1671429300000,3099.711946,17,0,0
4,1671429300000,0.8003,0.8009,0.8,0.8009,3349,1671429600000,2680.180335,31,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1671726600000,0.7764,0.7773,0.7756,0.7768,4869.5,1671726900000,3782.502702,26,0,0
996,1671726900000,0.7768,0.7776,0.7768,0.7768,2904.8,1671727200000,2257.606635,23,0,0
997,1671727200000,0.7768,0.7771,0.7761,0.7769,4001.55,1671727500000,3107.66736,31,0,0
998,1671727500000,0.7769,0.7781,0.7768,0.778,3407.29,1671727800000,2648.439327,29,0,0


In [47]:
def get5minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="MATICUSDT", interval="5m")["result"])
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [48]:
df = get5minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-19 05:35:00,0.8024,0.8024,0.8010,0.8021,9988.08
2022-12-19 05:40:00,0.8021,0.8021,0.8004,0.8009,15099.50
2022-12-19 05:45:00,0.8009,0.8028,0.8005,0.8028,6295.01
2022-12-19 05:50:00,0.8028,0.8028,0.8003,0.8003,3865.04
2022-12-19 05:55:00,0.8003,0.8009,0.8000,0.8009,3349.00
...,...,...,...,...,...
2022-12-22 16:30:00,0.7764,0.7773,0.7756,0.7768,4869.50
2022-12-22 16:35:00,0.7768,0.7776,0.7768,0.7768,2904.80
2022-12-22 16:40:00,0.7768,0.7771,0.7761,0.7769,4001.55


In [49]:
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)

In [50]:
apply_technicals(df)

In [51]:
df.head()

,Open,High,Low,Close,Volume,K,D,RSI
Time,,,,,,,,
2022-12-19 07:55:00,0.7985,0.7995,0.7978,0.7994,2673.22,0.184160,0.061387,40.198698
2022-12-19 08:00:00,0.7994,0.8017,0.7994,0.7999,9185.57,0.282207,0.155455,42.663938
2022-12-19 08:05:00,0.7999,0.8010,0.7994,0.7995,1558.05,0.224009,0.230125,41.200658
2022-12-19 08:10:00,0.7995,0.8001,0.7992,0.7995,6093.07,0.224009,0.243408,41.200658
2022-12-19 08:15:00,0.7995,0.8014,0.7987,0.8011,11992.37,0.646801,0.364940,49.271965


In [52]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) < 19)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Buy"]= np.where((self.df.trigger), 1, 0)


In [53]:
inst = Signals(df, 0)

In [54]:
inst.decide()

In [55]:
df[df.Buy==1]

,Open,High,Low,Close,Volume,K,D,RSI,trigger,Buy
Time,,,,,,,,,,
2022-12-19 22:30:00,0.7744,0.7746,0.756,0.7621,289837.03,0.0,0.020493,15.260835,1,1


In [56]:
#The mail addresses and password
sender_address = 'pythontradingbot11@gmail.com'
sender_pass = mail_key
receiver_address = mail
#Setup the MIME
message = MIMEMultipart() 
message_SL = MIMEMultipart()
message_TP = MIMEMultipart()
message_RSI = MIMEMultipart()
message_Others = MIMEMultipart()

In [57]:
def strategy_long(qty, open_position = False):
    df= get5minutedata()
    apply_technicals(df)
    inst = Signals(df, 0)
    inst.decide()
    print(f'Current Time is ' + str(df.index[-1]))
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    print(f'Current RSI is ' + str(df.RSI.iloc[-1]))
    print("-----------------------------------------")

    if df.Buy.iloc[-1]:
        mail_content = "Matic Open Long"
        message.attach(MIMEText(mail_content, 'plain'))
        
        # Create SMTP session for sending the mail
        session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
        session_mail.starttls()  # enable security

        # login with mail_id and password
        session_mail.login(sender_address, sender_pass)
        text = message.as_string()
        session_mail.sendmail(sender_address, receiver_address, text)
        session_mail.quit()

        from pybit import usdt_perpetual
        session = usdt_perpetual.HTTP(
        endpoint='https://api.bybit.com',
        api_key = api_key_pw,
        api_secret= api_secret_pw)

        buyprice = round(df.Close.iloc[-1],3)

        print("-----------------------------------------")

        print(f"Buyprice: {buyprice}")

        print("-----------------------------------------------------------------------------------------------------------------------------------------------")

        order = session.place_active_order(symbol="MATICUSDT",
                                                side="Buy",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=False,
                                                close_on_trigger=False,
                                                take_profit = round(buyprice * 1.06,3),
                                                stop_loss = round(buyprice * 0.98,3))
        print(order)

        matic_order_id = str(order['result']['order_id'])
        print("-----------------------------------------------------------------------------------------------------------------------------------------------")
        print(f"Order id: {matic_order_id}") 
        print("---------------------------------------------------")

        open_position = True

    while open_position:
        time.sleep(30)
                    
        df = get5minutedata()
        apply_technicals(df)
        print(f"Buyprice: {buyprice}" + '             Close: ' + str(df.Close.iloc[-1]))
        print(f'Target: ' + str(round(buyprice * 1.06,3)) + "                Stop: " + str(round(buyprice * 0.98,3)))
        print(f'RSI Target: 72' + '                RSI: ' + str(df.RSI.iloc[-1]))
        print("---------------------------------------------------")

        if df.Close[-1] < buyprice * 0.98: 
            print("Closed Position")
            open_position = False

            mail_content_SL = "Matic Long SL"
            message_SL.attach(MIMEText(mail_content_SL, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security

            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_SL.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break
        
        elif df.Close[-1] > buyprice* 1.06:
            print("Closed Position")
            open_position = False

            mail_content_TP = "Matic Long TP"
            message_TP.attach(MIMEText(mail_content_TP, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security

            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_TP.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break

        elif df.RSI[-1] > 72:
            session = usdt_perpetual.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)

            try:
                print(session.place_active_order(symbol="MATICUSDT",
                                                side="Sell",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=True,
                                                close_on_trigger=False)) 
                print("--------------------")           
                print("Closed position")
                open_position = False

                mail_content_RSI = "Matic Long Closed - RSI > 72"
                message_RSI.attach(MIMEText(mail_content_RSI, 'plain'))

                 # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_RSI.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break
            
            except: 
                print("Position already closed")
                
                mail_content_Others = "Position Closed"
                message_Others.attach(MIMEText(mail_content_Others, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_Others.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break

In [58]:
while True: 
    strategy_long(1200)
    time.sleep(60)

Current Time is 2022-12-22 16:50:00
Current Close is 0.778
Current RSI is 48.192650160960945
-----------------------------------------
